In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, 8, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(8, 2, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # self.conv3 = nn.Conv2d(4, 128, kernel_size=3, stride=1, padding=1)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(64, 32)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        # x = self.pool3(self.relu3(self.conv3(x)))
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu4(self.fc1(x))
        x = self.relu5(self.fc2(x))
        x = self.fc3(x) 

        return x

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import tqdm

In [6]:
data_dir = '/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/HumanvsLLM/HumanvsFlan/'

In [7]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

In [9]:
train_dataset = torchvision.datasets.ImageFolder(root=data_dir + '/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=data_dir + '/test', transform=transform)


In [10]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Step 3: Define the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(in_channels=3, num_classes=2).to(device)

In [11]:
print(test_dataset.find_classes(data_dir+'/test'))


(['Flan', 'Human'], {'Flan': 0, 'Human': 1})


In [13]:
import math
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, 1)
            TP += ((predicted == 1) & (labels == 1)).sum().item()
            FP += ((predicted == 1) & (labels == 0)).sum().item()
            TN += ((predicted == 0) & (labels == 0)).sum().item()
            FN += ((predicted == 0) & (labels == 1)).sum().item()
            
            total += labels.size(0)
            #print(labels, predicted)
            correct += (predicted == labels).sum().item()
    
    val_accuracy = correct / total
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))

    TPR = TP / (TP + FN)
    FPR = FP / (FP + TN)
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    print(f'TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}')
    print(f'Matthews Correlation Coefficient (MCC): {MCC:.4f}')
    print(f'True Positive Rate (TPR): {TPR:.4f}')
    print(f'False Positive Rate (FPR): {FPR:.4f}')



Epoch [1/10], Loss: 0.1895
Validation Accuracy: 0.9319
TP: 6092, FP: 218, TN: 7098, FN: 746
Matthews Correlation Coefficient (MCC): 0.8657
True Positive Rate (TPR): 0.8909
False Positive Rate (FPR): 0.0298
Epoch [2/10], Loss: 0.1707
Validation Accuracy: 0.9327
TP: 6342, FP: 456, TN: 6860, FN: 496
Matthews Correlation Coefficient (MCC): 0.8653
True Positive Rate (TPR): 0.9275
False Positive Rate (FPR): 0.0623
Epoch [3/10], Loss: 0.1621
Validation Accuracy: 0.9248
TP: 5850, FP: 76, TN: 7240, FN: 988
Matthews Correlation Coefficient (MCC): 0.8560
True Positive Rate (TPR): 0.8555
False Positive Rate (FPR): 0.0104
Epoch [4/10], Loss: 0.1563
Validation Accuracy: 0.9358
TP: 6056, FP: 126, TN: 7190, FN: 782
Matthews Correlation Coefficient (MCC): 0.8749
True Positive Rate (TPR): 0.8856
False Positive Rate (FPR): 0.0172
Epoch [5/10], Loss: 0.1511
Validation Accuracy: 0.9416
TP: 6330, FP: 318, TN: 6998, FN: 508
Matthews Correlation Coefficient (MCC): 0.8834
True Positive Rate (TPR): 0.9257
False

In [14]:
import torch
import torchvision.models as models

model_path = "/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/model/flan_model.pth"

torch.save(model.state_dict(), model_path)
